In [1]:
import pandas as pd
from glob import glob
import random
import Stemmer
import emoji
import re
from abc import ABC, abstractmethod
from num2words import num2words

In [120]:
appos = {
    "aren't" : "are not",
    "can't" : "cannot",
    "couldn't" : "could not",
    "didn't" : "did not",
    "doesn't" : "does not",
    "don't" : "do not",
    "hadn't" : "had not",
    "hasn't" : "has not",
    "haven't" : "have not",
    "he'd" : "he would",
    "he'll" : "he will",
    "he's" : "he is",
    "how's" : "how is",
    "i'd" : "i would",
    "i'll" : "i will",
    "i'm" : "i am",
    "i've" : "i have",
    "isn't" : "is not",
    "it's" : "it is",
    "let's" : "let us",
    "mustn't" : "must not",
    "shan't" : "shall not",
    "she'd" : "she would",
    "she'll" : "she will",
    "she's" : "she is",
    "shouldn't" : "should not",
    "that's" : "that is",
    "there's" : "there is",
    "they'd" : "they would",
    "they'll" : "they will",
    "they're" : "they are",
    "they've" : "they have",
    "wasn't" : "was not",
    "we'd" : "we would",
    "we'll" : "we will",
    "we're" : "we are",
    "we've" : "we have",
    "weren't" : "were not",
    "what's" : "what is",
    "where's" : "where is",
    "who's" : "who is",
    "won't" : "will not",
    "wouldn't" : "would not",
    "you'd" : "you would",
    "you'll" : "you will",
    "you're" : "you are",
    "you've" : "you have",
    "y'all" : "you all",
}

stop_words = {
    "i", "a", "an", "the", "in", "on", "at", "to", 
    "and", "or", "but", "is", "am", "are", "was", 
    "it", "they", "this", "of", "for", "with", 
    "as", "by", "from", "that", "those", "these",
    
    "he", "she", "we", "you", "me", "him", "her", 
    "us", "them", "my", "your", "our", "their", 
    "do", "does", "did", "have", "has", "had", 
    "about", "above", "across", "after", "against", 
    "along", "among", "around", "behind", "below", 
    "beneath", "beside", "between", "beyond", 
    "despite", "during", "except", "into", "near", 
    "off", "over", "past", "through", "toward", 
    "under", "until", "up", "upon", "via", "within", 
    "without", "although", "because", "since", 
    "unless", "while", "where", "whether", "any", 
    "each", "every", "either", "neither", "some", 
    "such", "both", "few", "many", "several", "all", 
    "other", "same", "so", "too","when"
}

negation_words = {
    "not", "no", "never", "none", "nobody", "nothing", "nowhere",
    "neither", "nor", "without", "cannot", "can't", "couldn't",
    "didn't", "doesn't", "don't", "hadn't", "hasn't", "haven't",
    "isn't", "mightn't", "mustn't", "needn't", "oughtn't", "shan't",
    "shouldn't", "wasn't", "weren't", "won't", "wouldn't", "rarely",
    "scarcely", "hardly", "barely", "little", "few", "lack", "lacking",
    "deny", "denies", "denied", "denying", "refuse", "refuses",
    "refused", "refusing", "reject", "rejects", "rejected", "rejecting"
}

positive_words = {
    "good", "great", "excellent", "awesome", "amazing", "wonderful",
    "fantastic", "perfect", "outstanding", "superb", "brilliant", "fabulous",
    "terrific", "incredible", "lovely", "delightful", "joyful", "happy",
    "joy", "bliss", "ecstatic", "pleasure", "vibrant", "optimistic",
    "positive", "success", "victory", "triumph", "peace", "harmony",
    "gratitude", "thankful", "blessed", "lucky", "prosperous", "rewarding",
    "satisfying", "refreshing", "inspiring", "motivating", "encouraging",
    "kind", "generous", "compassionate", "honest", "trustworthy", "loyal",
    "creative", "innovative", "energetic", "vital", "healthy", "strong",
    "courageous", "bold", "confident", "resilient", "graceful", "elegant",
    "charming", "friendly", "welcoming", "supportive", "helpful", "heavenly",
    "divine", "magical", "serene", "calm", "relaxed", "bright", "shining",
    "glowing", "sparkling", "clean", "pure", "fresh", "new", "exciting",
    "thrilling", "adventurous", "fun", "playful", "humorous", "lighthearted",
    "smiling", "laughing", "cheerful", "jubilant", "celebratory", "festive",
    "colorful", "beautiful", "stunning", "gorgeous", "radiant", "dazzling",
    "admire", "cherish", "treasure", "love", "adore", "passion", "devotion","smilingfacewithhearteyes"
}

negative_words = {
    "bad", "terrible", "awful", "horrible", "worst", "disgusting",
    "hate", "loathe", "despise", "abhor", "ugly", "painful",
    "sad", "unhappy", "miserable", "depressed", "gloomy", "sorrow",
    "failure", "disaster", "mess", "broken", "damaged", "defective",
    "angry", "furious", "enraged", "annoyed", "irritated", "frustrated",
    "evil", "wicked", "cruel", "mean", "selfish", "greedy", "corrupt",
    "harmful", "dangerous", "toxic", "poisonous", "deadly", "violent",
    "stupid", "idiotic", "foolish", "ridiculous", "nonsense", "absurd",
    "boring", "dull", "tedious", "monotonous", "lame", "uninteresting",
    "weak", "frail", "feeble", "helpless", "hopeless", "useless",
    "scary", "frightening", "terrifying", "horrifying", "creepy",
    "lonely", "isolated", "abandoned", "rejected", "betrayed", "ignored",
    "stress", "anxiety", "fear", "panic", "dread", "worry",
    "disappointment", "regret", "shame", "guilt", "embarrassment", "humiliation",
    "problem", "issue", "mistake", "error", "flaw", "defect",
    "conflict", "fight", "argument", "quarrel", "dispute", "war",
    "polluted", "dirty", "filthy", "rotten", "decaying", "waste",
    "fake", "fraud", "lie", "cheat", "deceive", "manipulate",
    "sick", "ill", "pain", "ache", "suffering", "agony",
    "loss", "defeat", "collapse", "crash", "bankrupt", "ruin"
}

positive_emojis = {
    ":grinning_face:", ":beaming_face_with_smiling_eyes:", ":grinning_face_with_sweat:",
    ":rolling_on_the_floor_laughing:", ":face_with_tears_of_joy:", ":smiling_face:",
    ":smiling_face_with_halo:", ":smiling_face_with_hearts:", ":smiling_face_with_heart-eyes:",
    ":star-struck:", ":kissing_face:", ":kissing_face_with_closed_eyes:",
    ":kissing_face_with_smiling_eyes:", ":heart_eyes:", ":face_blowing_a_kiss:",
    ":hugging_face:", ":thumbs_up:", ":clapping_hands:", ":folded_hands:", ":handshake:",
    ":sparkles:", ":fire:", ":rocket:", ":party_popper:", ":confetti_ball:", ":tada:",
    ":balloon:", ":red_heart:", ":orange_heart:", ":yellow_heart:", ":green_heart:",
    ":blue_heart:", ":purple_heart:", ":sparkling_heart:", ":heart_with_arrow:",
    ":revolving_hearts:", ":two_hearts:", ":growing_heart:", ":heartpulse:",
    ":star:", ":glowing_star:", ":sun:", ":sun_with_face:", ":rainbow:", ":flower_playing_cards:",
    ":cherry_blossom:", ":rose:", ":hibiscus:", ":sunflower:", ":tulip:", ":bouquet:",
    ":palm_tree:", ":christmas_tree:", ":four_leaf_clover:", ":shamrock:", ":maple_leaf:",
    ":fallen_leaf:", ":herb:", ":potted_plant:", ":seedling:", ":crown:", ":trophy:",
    ":medal:", ":musical_notes:", ":musical_note:", ":party_face:", ":dancer:", ":man_dancing:",
    ":woman_dancing:", ":man_in_tuxedo:", ":princess:", ":superhero:", ":rocket:", ":airplane:",
    ":checkered_flag:", ":trophy:", ":1st_place_medal:", ":money_with_wings:", ":dollar_banknote:",
    ":money_bag:", ":gem_stone:", ":ring:", ":gift:", ":birthday_cake:", ":champagne:",
    ":clinking_glasses:", ":beers:", ":chocolate_bar:", ":ice_cream:", ":doughnut:",
    ":cookie:", ":cake:", ":pizza:", ":hamburger:", ":taco:", ":burrito:", ":sushi:",
    ":ramen:", ":spaghetti:", ":wine_glass:", ":hot_beverage:", ":teacup_without_handle:",
    ":beer_mug:", ":clinking_beer_mugs:", ":champagne_glass:", ":dog_face:", ":cat_face:",
    ":panda_face:", ":koala:", ":unicorn_face:", ":dragon_face:", ":dove:", ":butterfly:",
    ":rainbow_flag:", ":peace_symbol:", ":infinity:", ":recycling_symbol:", ":white_flower:",
}

negative_emojis = {
    ":angry_face:", ":pouting_face:", ":face_with_symbols_on_mouth:", ":smiling_face_with_horns:",
    ":angry_face_with_horns:", ":skull:", ":skull_and_crossbones:", ":crying_face:",
    ":loudly_crying_face:", ":worried_face:", ":slightly_frowning_face:", ":confounded_face:",
    ":disappointed_face:", ":downcast_face:", ":face_with_steam_from_nose:", ":face_screaming_in_fear:",
    ":fearful_face:", ":anxious_face_with_sweat:", ":sad_but_relieved_face:", ":sleepy_face:",
    ":tired_face:", ":yawning_face:", ":face_with_medical_mask:", ":nauseated_face:",
    ":face_vomiting:", ":sneezing_face:", ":cold_face:", ":hot_face:", ":dizzy_face:",
    ":exploding_head:", ":face_with_head-bandage:", ":broken_heart:", ":black_heart:",
    ":thumbs_down:", ":middle_finger:", ":raised_fist:", ":oncoming_fist:", ":collision:",
    ":bomb:", ":fire:", ":dagger:", ":pistol:", ":stop_sign:", ":no_entry:", ":prohibited:",
    ":warning:", ":radioactive:", ":biohazard:", ":anger_symbol:", ":hole:", ":pile_of_poo:",
    ":zombie:", ":ghost:", ":japanese_goblin:", ":japanese_ogre:", ":clown_face:",
    ":crying_cat_face:", ":pouting_cat_face:", ":see-no-evil_monkey:", ":hear-no-evil_monkey:",
    ":speak-no-evil_monkey:", ":footprints:", ":spider:", ":spider_web:", ":snake:", ":rat:",
    ":scorpion:", ":mosquito:", ":microbe:", ":coffin:", ":funeral_urn:", ":stopwatch:",
    ":hourglass_done:", ":hourglass_not_done:", ":cigarette:", ":no_smoking:", ":cactus:",
    ":wilted_flower:", ":volcano:", ":earthquake:", ":tornado:", ":cloud_with_lightning_and_rain:",
    ":droplet:", ":sweat_droplets:", ":dashing_away:", ":dizzy:", ":face_with_thermometer:",
    ":sick:", ":bandage:", ":syringe:", ":pill:", ":test_tube:", ":magnifying_glass_tilted_left:",
    ":chains:", ":handcuffs:", ":locked:", ":locked_with_key:", ":locked_with_pen:"
}

# Create Load Data Class

In [50]:
class DataLoder:
    def __init__(self,path:str,labels:list[str],shuffle:bool=True):
        self.path = path
        self.labels = labels
        self.shuffle = shuffle
    
    def load_data(self)->pd.DataFrame:
        data=[]
        for label in self.labels:
            for filename in glob(f"{self.path}/{label}/*.txt"):
                with open(filename, "r", encoding="utf-8") as file:
                    text = file.read()
                    data.append({"text":text,"label":label})
        if self.shuffle:
            random.shuffle(data)
        df = pd.DataFrame(data)
        return df

In [51]:
data_load=DataLoder("data/training_data",labels=["neg","pos"],shuffle=True)
sentiment_data=data_load.load_data()
sentiment_data.head(5)

,text,label
0,nicolas cage comes up with an ingenious surviv...,neg
1,contact ( pg ) there's a moment late in robert...,pos
2,"after a rather disappointing "" mary railly "" ,...",pos
3,i want to correct what i wrote last year in my...,pos
4,when a film is produced on a shoestring budget...,neg


# Create Preprocessing Classes

In [66]:
class FeaturesExtractor(ABC):
    @abstractmethod
    def transform(self,text:str):
        pass

class Preprocessing(ABC):
    @abstractmethod
    def transform(self,text:str)->str:
        pass

In [124]:
class NegationCount(FeaturesExtractor):
    def transform(self,text:str):
        text=text.lower()
        tokens=text.split()
        counter=0
        for token in tokens:
            if token in negation_words:
                counter+=1
        return {
            "neg_word_count":counter
        }

class IngCount(FeaturesExtractor):
    def transform(self,text:str):
        text=RemoveSpecialCharacters().transform(text)
        tokens=text.split()
        counter=0
        for token in tokens:
            if token.endswith("ing"):
                counter+=1
        return {"ing_word_count":counter}

class EdCount(FeaturesExtractor):
    def transform(self,text:str):
        text=RemoveSpecialCharacters().transform(text)
        tokens=text.split()
        counter=0
        for token in tokens:
            if token.endswith("ed"):
                counter+=1
        return {"ed_word_count":counter}

class PositiveWordCount(FeaturesExtractor):
    def transform(self,text:str):
        text=RemoveSpecialCharacters().transform(text)
        tokens=text.split()
        counter=0
        for token in tokens:
            if token in positive_words:
                counter+=1
        return {"positive_word_count":counter}
            
class NegativeWordsCount(FeaturesExtractor):
    def transform(self,text:str):
        text=RemoveSpecialCharacters().transform(text)
        tokens=text.split()
        counter=0
        for token in tokens:
            if token in negative_words:
                counter+=1
        return {"negative_word_count":counter}

class PositiveEmojiCount(FeaturesExtractor):
    def transform(self,text:str):
        text=ConvertEmoji().transform(text)
        tokens=text.split()
        counter=0
        for token in tokens:
            if token.startswith(":") and token.endswith(":") and token in positive_emojis:
                counter+=1
        return {"positive_emoji_count":counter}

class NegativeEmojiCount(FeaturesExtractor):
    def transform(self,text:str):
        text=ConvertEmoji().transform(text)
        tokens=text.split()
        counter=0
        for token in tokens:
            print(token)
            if token.startswith(":") and token.endswith(":") and token in negative_emojis:
                counter+=1
        return {"negative_emoji_count":counter}
                

In [59]:
class RemoveSpecialCharacters(Preprocessing):
    def transform(self,text:str)->str:
        text=text.lower()
        process_text = re.sub(r"[^a-zA-Z0-9'\s]", '', text)
        process_text = re.sub(r"\s+", " ", process_text)
        return process_text

class ConvertEmoji(Preprocessing):    
    def transform(self,text:str)->str:
        text=text.lower()
        converted_text = emoji.demojize(text)
        return converted_text

class TextStemmer(Preprocessing):
    def transform(self,text:str)->str:
        text=text.lower()
        tokens=text.split()
        stemmer=Stemmer.Stemmer("english")
        tokens=[stemmer.stemWord(token) for token in tokens]
        return ' '.join(tokens)
    
class ConvertNumberToWords(Preprocessing):
    def transform(self,text:str)->str:
        text=text.lower()
        def replace_match(match):
            return num2words(int(match.group(0)), to='ordinal')
        return re.sub(r'-?\d+',replace_match,text)
        
class NegationHandling(Preprocessing):
    def transform(self,text:str)->str:
        text=text.lower()
        tokens=text.split()
        tokens=[appos[word] if word in appos else word for word in tokens]
        return ' '.join(tokens)

class RemoveStopWords(Preprocessing):
    def transform(self,text:str)->str:
        text=text.lower()
        tokens=text.split()
        tokens=[word for word in tokens if word not in stop_words]
        return ' '.join(tokens)

In [86]:
class Pipeline(ABC):
    @abstractmethod
    def transform(self,text:str):
        pass

In [87]:
class PreprocessingPipeline(Pipeline):
    def __init__(self,pipeline_list:list[Preprocessing]):
        self.pipeline_list = pipeline_list
    
    def transform(self,text:str):
        for pipeline in self.pipeline_list:
            text = pipeline.transform(text)
        
        return text

In [94]:
class FeaturesExtractorPipeline(Pipeline):
    def __init__(self,pipeline_list:list[FeaturesExtractor]):
        self.pipeline_list = pipeline_list
    
    def transform(self,text:str):
        dictionary ={}
        for pipeline in self.pipeline_list:
            dictionary.update(pipeline.transform(text))
        return dictionary            
            

In [125]:
string="100 I love this product! 😍 It's absolutely amazing. But the delivery was slow... 😠"
pipeline_pre=PreprocessingPipeline([
    NegationHandling(),
    TextStemmer(),
    ConvertEmoji(),
    RemoveSpecialCharacters(),
    ConvertNumberToWords(),
    RemoveStopWords()
])
pipeline_ext=FeaturesExtractorPipeline([
    NegationCount(),
    IngCount(),
    EdCount(),
    PositiveWordCount(),
    NegativeWordsCount(),
    PositiveEmojiCount(),
    NegativeEmojiCount(),
])
print(pipeline_pre.transform(string))
print(pipeline_ext.transform(string))

one hundredth love product smilingfacewithhearteyes absolut amazing deliveri slow angryface
100
i
love
this
product!
:smiling_face_with_heart-eyes:
it's
absolutely
amazing.
but
the
delivery
was
slow...
:angry_face:
{'neg_word_count': 0, 'ing_word_count': 1, 'ed_word_count': 0, 'positive_word_count': 2, 'negative_word_count': 0, 'positive_emoji_count': 1, 'negative_emoji_count': 1}
